In [970]:
import torch
import torch.nn as nn
import random
import glob
import string
import unicodedata

In [971]:
fileList = glob.glob('./data/names/*')

In [972]:
all_categories = [f.split('/')[-1].split('.')[0] for f in fileList]



In [973]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )



In [974]:
all_letters = string.ascii_letters + " .,;'-"
n_letters = len(all_letters) + 1

In [975]:
category_lines = {}
for fileName in fileList:
    cat = fileName.split('/')[-1].split('.')[0]
#     print(cat)
    names = []
    with open(fileName,'r') as f:
        for item in f:
            item = unicodeToAscii(item)
            names.append(item.lower())
    category_lines[cat] = names

In [976]:
category_size = len(all_categories)
input_size = n_letters
hidden_size = 128
output_size = n_letters
n_categories = category_size

In [961]:
class RNN1(nn.Module):
    def __init__(self, input_size, category_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(category_size + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(category_size + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)
        
        
        
    def forward(self, input, hidden, category):
        combined = torch.cat((input,hidden,category),1)
        output = self.i2o(combined)
        hidden = self.i2h(combined)
        out_combined = torch.cat((output,hidden),1)
        output = self.o2o(out_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        
        return output, hidden
            
    def initHidden(self):
        t = torch.zeros(1, self.hidden_size)
        return t
    

In [977]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden
    
    def initHidden(self):
        t = torch.zeros(1, self.hidden_size)
        return t    

In [978]:
def randomChoice1():
    cIx = random.randint(0, category_size-1)
    n_length = len(category_lines[all_categories[cIx]])
    nIx = random.randint(0, n_length-1)
    return all_categories[cIx], category_lines[all_categories[cIx]][nIx], n_length

In [979]:
cIx, nIx, n_length = randomChoice1()
print(cIx, nIx, n_length)

French tasse 277


In [980]:
# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line


In [991]:
def categoryTensor(category):
    t = torch.zeros(1, category_size)
    t[0][all_categories.index(category)] = 1
    return t

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    t = torch.zeros(len(line),1, n_letters)
    for i,l in enumerate(line):
        t[i][0][all_letters.find(l)] = 1
    return t

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    t = torch.zeros(len(line),1, len(all_letters))
    s =[]
    for i in range(1,len(line)):
        s.append(all_letters.find(line[i]))
    s.append(n_letters - 1)
    return torch.LongTensor(s)
#     return torch.LongTensor(letter_indexes)

In [992]:
def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor, input_line_tensor, target_line_tensor


def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor, input_line_tensor, target_line_tensor

In [993]:
category_tensor, input_line_tensor, target_line_tensor = randomTrainingExample()
target_line_tensor.unsqueeze_(-1)
print(category_tensor.size(), input_line_tensor.size(), target_line_tensor.size())

hidden = rnn.initHidden()
rnn.zero_grad()
loss = 0
output, hidden = rnn(category_tensor, input_line_tensor[0], hidden)

torch.Size([1, 18]) torch.Size([7, 1, 59]) torch.Size([7, 1])


In [997]:
criterion = nn.NLLLoss()

learning_rate = 0.0005

In [1124]:
for i in range(20000):
    category_tensor, input_line_tensor, target_line_tensor = randomTrainingExample()
    target_line_tensor.unsqueeze_(-1)

    hidden = rnn.initHidden()
    rnn.zero_grad()
    loss = 0

    for i,l in enumerate(input_line_tensor):
        output, hidden = rnn(category_tensor, l, hidden)
        loss += criterion(output, target_line_tensor[i])

    loss.backward()
    for par in rnn.parameters():
        p.data.add_(-learning_rate,p.grad.data)

print(loss.item()/input_line_tensor.size(0))    

4.040484428405762


In [1016]:
category_tensor, input_line_tensor, target_line_tensor = randomTrainingExample()
target_line_tensor.unsqueeze_(-1)
with torch.no_grad():
    hidden = rnn.initHidden()
    for i,l in enumerate(input_line_tensor):
        output, hidden = rnn(category_tensor, l, hidden)
        out_v, out_i = output.topk(1)
        print(all_letters[out_i])

    

h
G
-
-
-


In [1122]:
def sampleGen(category, start_letter='a'):
    with torch.no_grad():
        category_tensor = categoryTensor(category)
        input_letter = start_letter
        input = inputTensor(start_letter)
        max_len = 20
        hidden = rnn.initHidden()
        idx = 0
        output_name = start_letter
        
        while idx < max_len:
            idx +=1
            output, hidden = rnn(category_tensor, input[0], hidden)
            out_v, out_i = output.topk(1)
            topi = out_i[0][0]
            
#             print(out_i,out_i[0][0])
            output_name+=all_letters[out_i]
            input = inputTensor(all_letters[out_i])
            if topi == len(all_letters) -1:
                break
        
        return output_name
    
    
    

In [2]:
result = sampleGen('English','c')
# print(result)


In [1084]:
max_length = 20

# Sample from a category and starting letter
def sample(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name

In [1092]:
sample("English",'c')

'chchqhqhqhqMKPKf-KKKA'

In [994]:
criterion = nn.NLLLoss()

learning_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    return output, loss.item() / input_line_tensor.size(0)

In [995]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)



In [996]:
rnn = RNN(n_letters, 128, n_letters)

n_iters = 10
print_every = 5000
plot_every = 500
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0